# All WCS Points Eligible Tournaments

## Circuit

- Winter Regional Challengers
- Winter Championship
- Spring Regional Challengers
- Spring Championship
- Summer Regional Challengers
- Summer Championship
- Dreamhack Leipzig
- Gold Series International
- Dreamhack Austin
- Dreamhack Valencia
- IEM XI - Shanghai
- WCS Copa Intercontinental
- KeSPA Cup

## Korea

- GSL Season 1
- SSL Season 1
- Season 1 Cross finals
- GSL Season 2
- SSL Season 2
- Season 2 Cross finals
- KeSPA Cup

## Top-level Tournaments

- WCS 2016 - 52397
- GSL 2016 - 51442
- SSL 2016 - 51644

In [ ]:
# Friends don't let friends share api keys
import os
APIKEY = os.environ['ALIGULAC_KEY']
APIKEY

In [ ]:
import json
from urllib import request

API = 'http://aligulac.com/api/v1/'
DOMAIN = 'http://aligulac.com'

def fetch(path, qps = {}, silent=False):
    url = '{}{}?apikey={}&format=json'.format(API, path, APIKEY)
    if len(qps):
        url += '&' + '&'.join(( '{}={}'.format(key, qps[key]) for key in qps ))
    if not silent:
        print('---fetching: %s' % url)
    return json.loads(request.urlopen(url).read().decode('utf-8'))

def fetch_raw(url, qps = (), silent=False):
    if '?' not in url:
        initial_separator = '?'
    else:
        initial_separator = '&'
    full_url = '{}{}{}apikey={}&format=json'.format(DOMAIN, url, initial_separator, APIKEY)
    if len(qps):
        url += '&' + '&'.join(( '{}={}'.format(key, qps[key]) for key in qps ))
    if not silent:
        print('---fetching: %s' % full_url)
    return json.loads(request.urlopen(full_url).read().decode('utf-8'))

In [ ]:
# These are the three top-level nodes for all matches with WCS Points
event_ids = [
    52397, # WCS 2016
    51442, # GSL 2016
    51644  # SSL 2016
]

In [ ]:
import pandas as pd
import numpy as np

all_events = []

def fetch_child_events(res):
    children = res['children']
    if len(children):
        for child in children:
            fetch_child_events(fetch_raw(child, {}, True))
    else:
        print(res['fullname'])
        all_events.append(res)


In [ ]:
del all_events[:]
print('just checking: %d' % len(all_events))
for event in event_ids:
    fetch_child_events(fetch('event/%s' % event, {}, True))

In [ ]:
for event in all_events:
    event['parent_id'] = event['parent'].split('/')[-2]

events_pd = pd.read_json(json.dumps(all_events))
events_pd.head()

In [ ]:
filtered_events = events_pd[[ 'id', 'parent_id', 'earliest', 'latest', 'fullname', 'name' ]]
events_json = filtered_events.to_json(orient='records')
events_json

In [ ]:
with open('events.json', 'w') as f:
    f.write(json.dumps(json.loads(events_json), indent=2))